In [1]:
import pandas as pd

# Load the datasets
ETH_data_df = pd.read_csv("ETH-GBP.csv", parse_dates=['Date'])
global_markets_2020 = pd.read_csv("2020_Global_Markets_Data.csv", parse_dates=['Date'])
global_markets_2021 = pd.read_csv("2021_Global_Markets_Data.csv", parse_dates=['Date'])
global_markets_2022 = pd.read_csv("2022_Global_Markets_Data.csv", parse_dates=['Date'])
global_markets_2023 = pd.read_csv("2023_Global_Markets_Data.csv", parse_dates=['Date'])

# Organize datasets into a dictionary for easier access
datasets = {
    "ETH-Gold Data": ETH_data_df,
    "Global Markets 2020": global_markets_2020,
    "Global Markets 2021": global_markets_2021,
    "Global Markets 2022": global_markets_2022,
    "Global Markets 2023": global_markets_2023
}

# Display the head of each dataset to verify
dataset_heads = {name: df.head() for name, df in datasets.items()}
dataset_heads


{'ETH-Gold Data':         Date        Open        High         Low       Close   Adj Close  \
 0 2020-01-01   97.754478  100.102066   97.467186   98.714050   98.714050   
 1 2020-01-02   98.727661   98.727661   96.602150   96.912010   96.912010   
 2 2020-01-03   96.912827  102.866142   96.284966  102.573875  102.573875   
 3 2020-01-04  102.571434  104.011894  101.709099  103.260132  103.260132   
 4 2020-01-05  103.262215  106.521660  103.241982  104.199127  104.199127   
 
        Volume  
 0  5988583431  
 1  6109919641  
 2  8009516846  
 3  5680904209  
 4  5755001349  ,
 'Global Markets 2020':   Ticker       Date          Open          High           Low         Close  \
 0  ^NSEI 2020-01-01  12202.150391  12222.200195  12165.299805  12182.500000   
 1   ^NYA 2020-01-02  13913.030273  14003.389648  13913.030273  14002.490234   
 2  ^IXIC 2020-01-02   9039.459961   9093.429688   9010.889648   9092.190430   
 3  ^FTSE 2020-01-02   7542.399902   7624.799805   7542.399902   7604.299

In [2]:
# Combine the global market data for all years
combined_global_markets = pd.concat([global_markets_2020, global_markets_2021, global_markets_2022, global_markets_2023])

# Pivot the global market data to have dates as index, tickers as columns and Close prices as values
pivoted_global_markets = combined_global_markets.pivot(index='Date', columns='Ticker', values='Close')

# Merge the ETH-Gold data with the pivoted global market data
merged_data = ETH_data_df.merge(pivoted_global_markets, on='Date', how='left')

# Display the first few rows of the merged dataset
merged_data.head()


,Date,Open,High,Low,Close,Adj Close,Volume,000001.SS,CL=F,GC=F,^BSESN,^DJI,^FTSE,^GSPC,^IXIC,^N100,^N225,^NSEI,^NYA
0,2020-01-01,97.754478,100.102066,97.467186,98.714050,98.714050,5988583431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12182.500000,NaN
1,2020-01-02,98.727661,98.727661,96.602150,96.912010,96.912010,6109919641,3085.197998,61.180000,1524.500000,41626.640625,28868.800781,7604.299805,3257.850098,9092.190430,1158.260010,NaN,12282.200195,14002.490234
2,2020-01-03,96.912827,102.866142,96.284966,102.573875,102.573875,8009516846,3083.785889,63.049999,1549.199951,41464.609375,28634.880859,7622.399902,3234.850098,9020.769531,1157.380005,NaN,12226.650391,13917.049805
3,2020-01-04,102.571434,104.011894,101.709099,103.260132,103.260132,5680904209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-05,103.262215,106.521660,103.241982,104.199127,104.199127,5755001349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Adjust the columns to merge based on the correct column names in the gold_data
columns_to_merge = ['Date', 'Price', 'Vol.']

# Merge the Ethereum data with the converted gold data using the correct columns
merged_data = eth_data.merge(gold_data[columns_to_merge], on='Date', how='left', suffixes=('', '_gold'))

# Rename columns for clarity
merged_data.rename(columns={'Price': 'Close_gold', 'Vol.': 'Volume_gold'}, inplace=True)

# Handle any remaining missing values by forward filling
merged_data.fillna(method='ffill', inplace=True)

# Convert the 'Volume_gold' column to float after removing non-numeric characters (like 'M' or 'K')
merged_data['Volume_gold'] = merged_data['Volume_gold'].str.replace('M', '').str.replace('K', '').astype(float)

# Filter out the relevant columns
merged_data_new = merged_data[['Date', 'Close', 'Close_gold', 'Volume_gold']]

# Save the merged dataset to a new CSV file with a different name
file_path_new = "merged_eth_gold_data_v2.csv"
merged_data_new.to_csv(file_path_new, index=False)

merged_data_new.head()


,Date,Close,Close_gold,Volume_gold
0,2020-01-01,98.714050,NaN,NaN
1,2020-01-02,96.912010,"1,528.10",270.55
2,2020-01-03,102.573875,"1,552.40",436.74
3,2020-01-04,103.260132,"1,552.40",436.74
4,2020-01-05,104.199127,"1,552.40",436.74


In [5]:
# Handle NaN values by backward filling
merged_data_new.fillna(method='bfill', inplace=True)

# Check the first few rows to confirm the NaN value has been addressed
merged_data_new.head()


C:\Users\fabio\AppData\Local\Temp\ipykernel_7500\3723044183.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_new.fillna(method='bfill', inplace=True)


,Date,Close,Close_gold,Volume_gold
0,2020-01-01,98.714050,"1,528.10",270.55
1,2020-01-02,96.912010,"1,528.10",270.55
2,2020-01-03,102.573875,"1,552.40",436.74
3,2020-01-04,103.260132,"1,552.40",436.74
4,2020-01-05,104.199127,"1,552.40",436.74


In [7]:
# Load the newly uploaded dataset
eth_global_data_corrected = pd.read_csv('merged_eth_global_data_corrected.csv', parse_dates=['Date'])



# Ensure the 'Date' columns in both dataframes have the same datetime type
eth_global_data_corrected['Date'] = pd.to_datetime(eth_global_data_corrected['Date'])
merged_data_new['Date'] = pd.to_datetime(merged_data_new['Date'])

# Merge the data again
final_merged_data = merged_data_new.merge(eth_global_data_corrected, on='Date', how='left', suffixes=('', '_global'))

# Save the final merged dataset to a new CSV file
file_path_final = "final_merged_eth_gold_global_data_v1.csv"
final_merged_data.to_csv(file_path_final, index=False)

final_merged_data.head()


C:\Users\fabio\AppData\Local\Temp\ipykernel_7500\1823624809.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_new['Date'] = pd.to_datetime(merged_data_new['Date'])


,Date,Close,Close_gold,Volume_gold,Unnamed: 0,Open,High,Low,Close_global,Adj Close,...,GC=F,^BSESN,^DJI,^FTSE,^GSPC,^IXIC,^N100,^N225,^NSEI,^NYA
0,2020-01-01,98.714050,"1,528.10",270.55,0,97.754478,100.102066,97.467186,98.714050,98.714050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12182.500000,NaN
1,2020-01-02,96.912010,"1,528.10",270.55,1,98.727661,98.727661,96.602150,96.912010,96.912010,...,1524.500000,41626.640625,28868.800781,7604.299805,3257.850098,9092.190430,1158.260010,NaN,12282.200195,14002.490234
2,2020-01-03,102.573875,"1,552.40",436.74,2,96.912827,102.866142,96.284966,102.573875,102.573875,...,1549.199951,41464.609375,28634.880859,7622.399902,3234.850098,9020.769531,1157.380005,NaN,12226.650391,13917.049805
3,2020-01-04,103.260132,"1,552.40",436.74,3,102.571434,104.011894,101.709099,103.260132,103.260132,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-05,104.199127,"1,552.40",436.74,4,103.262215,106.521660,103.241982,104.199127,104.199127,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Saving the merged data to a CSV file
merged_data_path = "final_merged_eth_gold_global_data_v1.csv"
merged_data.to_csv(merged_data_path)

merged_data_path


In [ ]:
# Convert 'Close_gold' column to numeric, as it seems to be in string format with commas
merged_data['Close_gold'] = pd.to_numeric(merged_data['Close_gold'].str.replace(',', ''), errors='coerce')

# Check for inf values in the 'Close_gold' and 'Volume_gold' columns
inf_values_close_gold = np.isinf(merged_data['Close_gold']).sum()
inf_values_volume_gold = np.isinf(merged_data['Volume_gold']).sum()

# Replace inf values with NaN
merged_data['Close_gold'].replace([np.inf, -np.inf], np.nan, inplace=True)
merged_data['Volume_gold'].replace([np.inf, -np.inf], np.nan, inplace=True)

inf_values_close_gold, inf_values_volume_gold
